<h1 align="center">Decission Tree Project</h1>
<h4 align="center">Shaji Joseph</h4>
<h4 align="center"> CSCI 513: Python Programming for AI</h4>
<h4 align="center"> CWID: 50394653</h4>

In [ ]:
# Pre-Requisite Libraries

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, cross_val_predict
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from scipy import sparse
from sklearn.feature_selection import SelectKBest, chi2, VarianceThreshold

In [ ]:


                                  ###################
                                  #  Data Load      #
                                  ###################
data = pd.read_excel('Building Permits.xlsx')
print(data.head())

In [ ]:



                                    #############################
                                    # Pre-Processing the Data   #
                                    #############################
# Rename the Column Names
rename_dict = {
    'ISSINGLEFAMILY': 'SINGLE FAMILY',
    'ISMULTIFAMILY': 'MULTI FAMILY',
    'ISCOMMERCIAL': 'COMMERCIAL',
    'GISDISTRICT': 'GIS DISTRICT',
    'CREATEDDATE': 'CREATED DATE',
    'COMPLETEDDATE': 'COMPLETED DATE',
    'CONTRUCTIONTIME': 'CONSTRUCTION TIME',
    'NEWCONSTRUCTION': 'NEW CONSTRUCTION',
    'EXPRESSPLANREVIEW': 'EXPRESS PLAN REVIEW',
    'PERMITTYPE2LETTER': 'PERMIT TYPE',
    'JOBVALUE': 'JOB VALUE',
    'TOTALSQUAREFEET': 'TOTAL SQUARE FEET',
    'WORKDESCRIPTION': 'WORK DESCRIPTION',
    'CONTRACTORTYPE': 'CONTRACTOR TYPE',
    'NEWCONSTRUCTIONCOST': 'NEW CONSTRUCTION COST',
    'REMODELCONSTRUCTIONCOST': 'REMODEL CONSTRUCTION COST'
    
}
data = data.rename(columns=rename_dict)

# Formating the Dates to proper datetime format
data['CREATED DATE'] = pd.to_datetime(data['CREATED DATE'], format='mixed')
data['COMPLETED DATE'] = pd.to_datetime(data['COMPLETED DATE'], format='mixed')
data['CONSTRUCTION TIME'] = (data['COMPLETED DATE'] - data['CREATED DATE']).dt.days

# Map the Binary Column Values to 1 or 0 (Y=1, N=0, Other=0)

booleancols = ['SINGLE FAMILY', 'MULTI FAMILY', 'COMMERCIAL', 'NEW CONSTRUCTION', 'REMODEL', 'ADDITION', 'EXPRESS PLAN REVIEW']
for col in booleancols:
    if col in data.columns:
        data[col] = np.where(data[col] == 'Y', 1, 0)

# Droping unwanted columns from the Dataframe
todropcols = ['OWNERADDRESS', 'OWNERNAME', 'OWNERCODE', 'APPLICANTNAME', 'ISSUEDATE', 'EXTERNALFILENUM', 'ADDRESSOBJECTID', 'PROJECTDESCRIPTION', 'DESCRIPTION', 'TRADELIST',
    'EXPRESSPLANREVIEWFEE', 'CONTRACTORPHONE', 'WORKCODEPRINTED','ISREVIEWAPPROVED', 'WORKCODE', 'GISCOUNCILDISTRICT']
data.drop(columns=[col for col in todropcols if col in data.columns], inplace=True)

# Additional Data Cleaning
clncols = ['PERMIT TYPE', 'ACTIVITY', 'WORK DESCRIPTION', 'GIS DISTRICT']
clncols = [col for col in clncols if col in data.columns]
data[clncols] = data[clncols].apply(lambda x: x.str.strip().str.upper().fillna('UNKNOWN'))

# Handling Non-numeric Values by updating them with 0
numcols = ['JOB VALUE', 'TOTAL SQUARE FEET']
numcols = [col for col in numcols if col in data.columns]
data[numcols] = data[numcols].apply(pd.to_numeric, errors='coerce').fillna(0)

# Decade Calculation and Create a new column for Residential Building Type
data['YEAR'] = data['CREATED DATE'].dt.year
data['DECADE'] = (data['YEAR'] // 10) * 10
data['IS RESIDENTIAL'] = (data['SINGLE FAMILY'] | data['MULTI FAMILY']).astype(int)

# Number of Permits were issued per decade for each building type
group_decade = data.groupby('DECADE')
commercial = group_decade['COMMERCIAL'].sum()
singlefamily = group_decade['SINGLE FAMILY'].sum()
multifamily = group_decade['MULTI FAMILY'].sum()

In [ ]:
                    #######################################################
                    # Count of Permits by Decades for Building Permits    #
                    #######################################################
# Create decade labels
decadelabels = [str(decade) + '-' + str(decade + 9) for decade in singlefamily.index]

# Plot Stacked Bar Chart
fig = go.Figure(go.Bar(x=decadelabels, y=singlefamily.values, name='SINGLE FAMILY'))
fig.add_trace(go.Bar(x=decadelabels, y=multifamily.values, name='MULTI FAMILY'))
fig.add_trace(go.Bar(x=decadelabels, y=commercial.values, name='COMMERCIAL'))

fig.update_layout(barmode='stack', xaxis_title='Decade', yaxis_title='Number of Permits', title='Building Permit Types by Decade', template='plotly_white')
fig.show()

In [ ]:
                        #############################################################
                        # Average Construction Time for Building Projects by Year   #
                        #############################################################
# Filter for BU (Building) permits
bu_data = data[(data['PERMIT TYPE'] == 'BU') & (data['COMPLETED DATE'].dt.year >= 1975) & (data['COMPLETED DATE'].dt.year <= 2024)]

# Get average construction time and counts per completed year
def get_stats(df, building_type):
    subset = df[df[building_type] == 1]
    grouped = subset.groupby(df['COMPLETED DATE'].dt.year)['CONSTRUCTION TIME'].agg(['mean', 'count']).reset_index()
    grouped.columns = ['YEAR', 'AVG_TIME', 'COUNT']
    return grouped

# Get stats for each type (Single Family, Multi Family, and Commercial)
sf_stats = get_stats(bu_data, 'SINGLE FAMILY')
mf_stats = get_stats(bu_data, 'MULTI FAMILY')
com_stats = get_stats(bu_data, 'COMMERCIAL')

# Plot using Plotly
fig = go.Figure()

# Single Family
fig.add_trace(go.Scatter(x=sf_stats['YEAR'], y=sf_stats['AVG_TIME'], mode='lines+markers', name='Single Family', line=dict(color='royalblue'), customdata=sf_stats['COUNT'],
hovertemplate='Year: %{x}<br>Avg Time: %{y:.1f} days<br>Permits: %{customdata}<extra></extra>'))

# Multi Family
fig.add_trace(go.Scatter(x=mf_stats['YEAR'], y=mf_stats['AVG_TIME'], mode='lines+markers', name='Multi Family', line=dict(color='darkorange'), customdata=mf_stats['COUNT'],
    hovertemplate='Year: %{x}<br>Avg Time: %{y:.1f} days<br>Permits: %{customdata}<extra></extra>'))

# Commercial
fig.add_trace(go.Scatter(x=com_stats['YEAR'], y=com_stats['AVG_TIME'], mode='lines+markers', name='Commercial', line=dict(color='firebrick'), customdata=com_stats['COUNT'],
    hovertemplate='Year: %{x}<br>Avg Time: %{y:.1f} days<br>Permits: %{customdata}<extra></extra>'))

# Final layout
fig.update_layout(title='Average Construction Time by Year for BU Permits (with Permit Counts)', xaxis_title='Year', yaxis_title='Average Construction Time (Days)', template='presentation')

fig.show()


In [ ]:
                                    ###########################################
                                    #   PLOT JOB COUNT PER DECADE BY ZIP CODE #
                                    ##########################################
# Create DECADE LABEL column
data['DECADE LABEL'] = data['DECADE'].astype(str) + '–' + (data['DECADE'] + 9).astype(str)

# Convert ZIPCODE to string and remove decimals
data['ZIPCODE'] = data['ZIPCODE'].astype(str).str.extract(r'(\d+)')[0]

# Group by DECADE LABEL and ZIPCODE
group = data.groupby(['DECADE LABEL', 'ZIPCODE']).size().reset_index(name='Job Count')

# Plot grouped bar chart
fig = px.bar(group, x='DECADE LABEL', y='Job Count', color='ZIPCODE', barmode='group', color_discrete_sequence=px.colors.qualitative.Dark24, category_orders={'DECADE LABEL': sorted(group['DECADE LABEL'].unique())},
    title='Job Count per Decade by ZIP Code')

fig.update_layout(bargap=0.3, bargroupgap=0.1, xaxis_title='Decade', yaxis_title='Job Count', template='plotly_white')

fig.show()


In [ ]:
                                    #################################
                                    #   PLOT JOB COUNT PER DECADE   #
                                    #################################
# Group by DECADE LABEL and ZIPCODE
fig = px.box(group, x='DECADE LABEL', y='Job Count')
# Plot Box and Whisker Plot chart
fig.update_layout(xaxis_title='Decade', yaxis_title='Job Count', title='Job Count Distribution by Decade')
fig.show()

In [ ]:
# Combine multiple binary columns into one multi-class label
def label_type(row):
    if row['COMMERCIAL'] == 1:
        return 'COMMERCIAL'
    elif row['MULTI FAMILY'] == 1:
        return 'MULTI FAMILY'
    elif row['SINGLE FAMILY'] == 1:
        return 'SINGLE FAMILY'
    else:
        return 'OTHER'
data['PROPERTY TYPE'] = data.apply(label_type, axis=1)

In [ ]:
features = ['CONSTRUCTION TIME','YEAR','ZIPCODE','NEW CONSTRUCTION COST', 'REMODEL CONSTRUCTION COST','TOTAL SQUARE FEET','JOB VALUE']
X = data[features]
y = data['PROPERTY TYPE']

In [ ]:
print("Features (X):", X.shape)
print("Labels (y):", y.shape)
print("Features:")
print(X.head(5))
print("Labels:")
print(y.head(5))

In [ ]:
                                        ##############################################         
                                        # Multiclass Classification - Decission Tree #
                                        # Holdout Evaluation                         #
                                        ##############################################
# Train-test split - Holdout Method
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the DecisionTreeClassifier model
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))
# Print Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(cm)

In [ ]:
# Define the label Order
labels = ['COMMERCIAL', 'MULTI FAMILY', 'SINGLE FAMILY']  
# Create DataFrame for Plotly
cm_df = pd.DataFrame(cm, index=labels, columns=labels)
# Plot using Plotly
fig = px.imshow(cm_df, text_auto=True, color_continuous_scale='BrBG', labels=dict(x="Predicted", y="Actual", color="Count"), x=labels, y=labels)
fig.update_layout(title="Confusion Matrix", width=700, height=700)
fig.show()

In [ ]:
                                            ###############################################
                                            # Multiclass Classification - Decission Tree  #
                                            # Evaluation - Cross Validation               #
                                            ###############################################
# X as a DataFrame
X = pd.DataFrame(X)

# Keep numeric columns, fill NaNs, and replace negative values
numeric_cols = X.select_dtypes(include=[np.number]).columns
X_numeric = X[numeric_cols].fillna(0).clip(lower=0)

# Convert to sparse matrix
X_sparse = sparse.csr_matrix(X_numeric.values)

# Remove near-constant features
X_sparse_reduced = VarianceThreshold(threshold=1e-5).fit_transform(X_sparse)

# Select top k features using chi2
X_sparse_reduced = SelectKBest(chi2, k=6).fit_transform(X_sparse_reduced, y)

# Define classifier
clf = DecisionTreeClassifier(random_state=42)

# Define cross-validation Method
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Cross-validation accuracy scores
scores = cross_val_score(clf, X_sparse_reduced, y, cv=cv, scoring='accuracy')
print("Cross-Validation Accuracy: Mean = {:.3f}, Std Dev = {:.3f}".format(scores.mean(), scores.std()))


# Predict using cross-validation
y_pred_cv = cross_val_predict(clf, X_sparse_reduced, y, cv=cv)

# Per-Property Type metrics from cross-validation
print("\nCross-Validation Classification Report (per Property Type):")
print(classification_report(y, y_pred_cv))

# Confusion Matrix from cross-validation
print("Cross-Validation Confusion Matrix:")
print(confusion_matrix(y, y_pred_cv))


In [ ]:

# Define the label Order
targetlabels = ['COMMERCIAL', 'MULTI FAMILY', 'SINGLE FAMILY']

# Generate Confusion Matrix
cm = confusion_matrix(y, y_pred_cv, labels=targetlabels)

# Create DataFrame for Plotly
cm_df = pd.DataFrame(cm, index=targetlabels, columns=targetlabels)

# Plot using Plotly
fig = px.imshow(cm_df, text_auto=True, color_continuous_scale='Spectral', labels=dict(x="Predicted", y="Actual", color="Count"), x=targetlabels, y=targetlabels)

fig.update_layout(title="Cross-Validation Confusion Matrix", width=700, height=700)
fig.show()
